Author: Sid Ali Rezetane
In This Notebook, I have gathered several techniques, namely : Guassian Naive bayes model (probabilistic classifier), Logistic regression with PCA Decomp, Keras Classifier (Deep learning) with Stochastic gradient descent, and Grid search. 
they are run according to the following sequence : 
   -- GaussianNB Model 
   -- LogisticRegression PCA
   -- Keras Classifer SGD (Wrapper)
   -- Grid Search (different Epochs and Batch Sizes)


Data Used : Labels and Metacodes ( Binary Values only ) 
42 Possible Metacodes, 36 used ( some codes are pure opposite of others ) 
Load the whole data [ datetime, lables, prices and patterns } Use only Labels and Patterns ! 

Gaussian Naive Bayes Classifier

Load the libraries 

In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics  import accuracy_score
import preprocessing
import series

load the file hat contains data, specify split ration (85% train 15% Test)

In [8]:
filename = 'provametacodes.csv'
split_ratio = 0.85
window_size =1
x , y = series.importing(filename)
#not for gaussianNB
#x , y = series.windowing(x,y,window_size)
train_x,train_y,test_x,test_y = series.split_x_y(x,y,split_ratio)

Build the model, fit and predict then compute accuracy 

In [9]:
#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets 
model.fit(train_x, train_y)

#Predict Output 
predicted= model.predict(test_x)
accuracy = accuracy_score(test_y,predicted,normalize=True)
#error2  = preprocessing.accuracy(test_y,predicted)
print("accuracy= ",accuracy*100)
#print("",predicted)

accuracy=  61.3488975357


In [ ]:
%reset


KERAS Classifier with SGD and Cross validation

Load libraries

In [ ]:

# Baseline Model Keras Classifier SGD with Cross validation score
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import series
from keras.layers import LSTM

Load file, specify split ratio (85% training Data, 15% Testing data)

In [ ]:

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
filename = 'provametacodes.csv'
split_ratio = 0.85
window_size =1
x , y = series.importing(filename)

#x , y = series.windowing(x,y,window_size)
train_x,train_y,test_x,test_y = series.split_x_y(x,y,split_ratio)
y = numpy.asarray(y)
#y = numpy.reshape(y, (y.shape[0],1,1))

x = numpy.array(x)
#x = numpy.reshape(x, (x.shape[0],1, x.shape[1]))

Build Model, set the optimizer to SGD 
Build the pipeline 

In [ ]:
def create_baseline():
	# create model
    model = Sequential()
 #   model.add(LSTM(64, input_shape=[1,36], return_sequences = True))
    model.add(Dense(120, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
    sgd = SGD(lr=0.1, momentum=0.8, decay=0.1, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
 
numpy.random.seed(seed)
estimators = []
#estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=16, verbose=2)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy : 62.02%

In [14]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Logistic Regression using PCA Decomposition to select best features (features that contributes most to target)
Load libraries

In [15]:
# Create a pipeline that extracts features from the data then creates a model
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
import series
import numpy

load data and split 

In [16]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
filename = 'provametacodes.csv'
split_ratio = 0.75
window_size =1
x , y = series.importing(filename)

#x , y = series.windowing(x,y,window_size)
train_x,train_y,test_x,test_y = series.split_x_y(x,y,split_ratio)

Decompose to n_components, select features build the pipline and compute the cross validation score

In [17]:
# create feature union
features = []
features.append(('pca', PCA(n_components=2)))
features.append(('select_best', SelectKBest(k=10)))
feature_union = FeatureUnion(features)
# create pipeline
estimators = []
estimators.append(('feature_union', feature_union))
estimators.append(('logistic', LogisticRegression()))
model = Pipeline(estimators)
# evaluate pipeline
seed = 7
kfold = KFold(n_splits=6, random_state=seed)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean(),results.std())

0.61981978175 0.0341851842157


Accuracy : 61.98% 

In [18]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Grid Search using keras classifier, try and select best accuracy with this grid : epochs = [5,10,20] batches = [50,100,200] Optimizer = [rmsprop, adam]

In [19]:

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy
import series

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
filename = 'provametacodes.csv'
split_ratio = 0.75
window_size =1
x , y = series.importing(filename)
train_x,train_y,test_x,test_y = series.split_x_y(x,y,split_ratio)

build the classifier, set the grid { Optimizer, init, epoch and batches }

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=x.shape[1], kernel_initializer=init, activation='relu'))
	model.add(Dense(8, kernel_initializer=init, activation='relu'))
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [5,10,20]
batches = [50,100,200]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/5
3424/3424 [==============================] - 0s 124us/step - loss: 0.6687 - acc: 0.6276
Epoch 2/5
3424/3424 [==============================] - ETA: 0s - loss: 0.6612 - acc: 0.630 - 0s 35us/step - loss: 0.6623 - acc: 0.6279
Epoch 3/5
3424/3424 [==============================] - 0s 28us/step - loss: 0.6592 - acc: 0.6273
Epoch 4/5
3424/3424 [==============================] - 0s 24us/step - loss: 0.6565 - acc: 0.6282
Epoch 5/5
3424/3424 [==============================] - 0s 13us/step
Epoch 1/5
3425/3425 [==============================] - 1s 162us/step - loss: 0.6848 - acc: 0.5764
Epoch 2/5
3425/3425 [==============================] - 0s 25us/step - loss: 0.6643 - acc: 0.6251
Epoch 3/5
3425/3425 [==============================] - ETA: 0s - loss: 0.6642 - acc: 0.618 - 0s 23us/step - loss: 0.6606 - acc: 0.6254
Epoch 4/5
3425/3425 [==============================] - 0s 25us/step - loss: 0.6588 - acc: 0.6254
Epoch 5/5
3425/3425 [==============================] - 0s 15us/step
Epoch 1/5


3425/3425 [==============================] - 0s 24us/step
Epoch 1/10
3425/3425 [==============================] - 1s 310us/step - loss: 0.6802 - acc: 0.6015
Epoch 2/10
3425/3425 [==============================] - 0s 26us/step - loss: 0.6714 - acc: 0.6076
Epoch 3/10
3425/3425 [==============================] - 0s 25us/step - loss: 0.6672 - acc: 0.6076
Epoch 4/10
3425/3425 [==============================] - 0s 23us/step - loss: 0.6648 - acc: 0.6082
Epoch 5/10
3425/3425 [==============================] - 0s 23us/step - loss: 0.6627 - acc: 0.6085
Epoch 6/10
3425/3425 [==============================] - 0s 22us/step - loss: 0.6615 - acc: 0.6088
Epoch 7/10
3425/3425 [==============================] - 0s 36us/step - loss: 0.6602 - acc: 0.6091
Epoch 8/10
3425/3425 [==============================] - 0s 30us/step - loss: 0.6593 - acc: 0.6091
Epoch 9/10
3425/3425 [==============================] - 0s 54us/step - loss: 0.6581 - acc: 0.6105
Epoch 10/10
3425/3425 [==============================] - 0s

Epoch 20/20
3424/3424 [==============================] - 0s 18us/step
Epoch 1/20
3425/3425 [==============================] - 2s 515us/step - loss: 0.6829 - acc: 0.5909
Epoch 2/20
3425/3425 [==============================] - 0s 39us/step - loss: 0.6654 - acc: 0.6274
Epoch 3/20
3425/3425 [==============================] - 0s 40us/step - loss: 0.6598 - acc: 0.6254
Epoch 4/20
3425/3425 [==============================] - 0s 38us/step - loss: 0.6587 - acc: 0.6254
Epoch 5/20
3425/3425 [==============================] - 0s 27us/step - loss: 0.6573 - acc: 0.6254
Epoch 6/20
3425/3425 [==============================] - 0s 38us/step - loss: 0.6563 - acc: 0.6254
Epoch 7/20
3425/3425 [==============================] - 0s 26us/step - loss: 0.6551 - acc: 0.6248
Epoch 8/20
3425/3425 [==============================] - 0s 25us/step - loss: 0.6544 - acc: 0.6260
Epoch 9/20
3425/3425 [==============================] - 0s 26us/step - loss: 0.6536 - acc: 0.6269
Epoch 10/20
3425/3425 [========================

3425/3425 [==============================] - 0s 32us/step - loss: 0.6441 - acc: 0.6315
Epoch 19/20
3425/3425 [==============================] - 0s 41us/step - loss: 0.6422 - acc: 0.6385
Epoch 20/20
3425/3425 [==============================] - 0s 28us/step
Epoch 1/20
3425/3425 [==============================] - 2s 490us/step - loss: 0.6794 - acc: 0.5801
Epoch 2/20
3425/3425 [==============================] - 0s 51us/step - loss: 0.6683 - acc: 0.6067
Epoch 3/20
3425/3425 [==============================] - 0s 34us/step - loss: 0.6652 - acc: 0.6073
Epoch 4/20
3425/3425 [==============================] - 0s 30us/step - loss: 0.6636 - acc: 0.6061
Epoch 5/20
3425/3425 [==============================] - 0s 33us/step - loss: 0.6622 - acc: 0.6093
Epoch 6/20
3425/3425 [==============================] - 0s 33us/step - loss: 0.6612 - acc: 0.6055
Epoch 7/20
3425/3425 [==============================] - 0s 31us/step - loss: 0.6605 - acc: 0.6082
Epoch 8/20
3425/3425 [==============================] - 0

3424/3424 [==============================] - 0s 36us/step - loss: 0.6524 - acc: 0.6276
Epoch 15/20
3424/3424 [==============================] - 0s 33us/step - loss: 0.6519 - acc: 0.6276
Epoch 16/20
3424/3424 [==============================] - 0s 51us/step - loss: 0.6516 - acc: 0.6276
Epoch 17/20
3424/3424 [==============================] - 0s 31us/step - loss: 0.6512 - acc: 0.6276
Epoch 18/20
3424/3424 [==============================] - 0s 30us/step - loss: 0.6507 - acc: 0.6276
Epoch 19/20
3424/3424 [==============================] - 0s 33us/step - loss: 0.6506 - acc: 0.6276
Epoch 20/20
3424/3424 [==============================] - 0s 20us/step
Epoch 1/20
3425/3425 [==============================] - 2s 474us/step - loss: 0.6804 - acc: 0.6254
Epoch 2/20
3425/3425 [==============================] - 0s 33us/step - loss: 0.6630 - acc: 0.6254
Epoch 3/20
3425/3425 [==============================] - 0s 33us/step - loss: 0.6610 - acc: 0.6254
Epoch 4/20
3425/3425 [==============================]

...

3424/3424 [==============================] - 0s 30us/step - loss: 0.6614 - acc: 0.6276
Epoch 4/20
3424/3424 [==============================] - 0s 37us/step - loss: 0.6592 - acc: 0.6276
Epoch 5/20
3424/3424 [==============================] - 0s 30us/step - loss: 0.6582 - acc: 0.6276
Epoch 6/20
3424/3424 [==============================] - 0s 31us/step - loss: 0.6572 - acc: 0.6276
Epoch 7/20
3424/3424 [==============================] - 0s 27us/step - loss: 0.6562 - acc: 0.6276
Epoch 8/20
3424/3424 [==============================] - 0s 31us/step - loss: 0.6556 - acc: 0.6276
Epoch 9/20
3424/3424 [==============================] - 0s 25us/step - loss: 0.6548 - acc: 0.6276
Epoch 10/20
3424/3424 [==============================] - 0s 45us/step - loss: 0.6543 - acc: 0.6276
Epoch 11/20
3424/3424 [==============================] - 0s 28us/step - loss: 0.6538 - acc: 0.6276
Epoch 12/20
3424/3424 [==============================] - 0s 28us/step - loss: 0.6528 - acc: 0.6276
Epoch 13/20
3424/3424 [=====

3425/3425 [==============================] - 4s 1ms/step - loss: 0.6921 - acc: 0.5921
Epoch 2/5
3425/3425 [==============================] - 0s 25us/step - loss: 0.6887 - acc: 0.6076
Epoch 3/5
3425/3425 [==============================] - 0s 22us/step - loss: 0.6824 - acc: 0.6076
Epoch 4/5
3425/3425 [==============================] - 0s 16us/step - loss: 0.6749 - acc: 0.6076
Epoch 5/5
3425/3425 [==============================] - 0s 14us/step
Epoch 1/10
3424/3424 [==============================] - 4s 1ms/step - loss: 0.7010 - acc: 0.4536
Epoch 2/10
3424/3424 [==============================] - 0s 15us/step - loss: 0.6792 - acc: 0.5806
Epoch 3/10
3424/3424 [==============================] - 0s 18us/step - loss: 0.6705 - acc: 0.6273
Epoch 4/10
3424/3424 [==============================] - 0s 15us/step - loss: 0.6649 - acc: 0.6282
Epoch 5/10
3424/3424 [==============================] - 0s 15us/step - loss: 0.6607 - acc: 0.6285
Epoch 6/10
3424/3424 [==============================] - 0s 17us/st

3425/3425 [==============================] - 0s 15us/step - loss: 0.6693 - acc: 0.6076
Epoch 6/10
3425/3425 [==============================] - 0s 16us/step - loss: 0.6688 - acc: 0.6076
Epoch 7/10
3425/3425 [==============================] - 0s 15us/step - loss: 0.6681 - acc: 0.6076
Epoch 8/10
3425/3425 [==============================] - 0s 15us/step - loss: 0.6677 - acc: 0.6076
Epoch 9/10
3425/3425 [==============================] - 0s 15us/step - loss: 0.6673 - acc: 0.6076
Epoch 10/10
3425/3425 [==============================] - 0s 11us/step
Epoch 1/10
3424/3424 [==============================] - 4s 1ms/step - loss: 0.6920 - acc: 0.6183
Epoch 2/10
3424/3424 [==============================] - 0s 19us/step - loss: 0.6884 - acc: 0.6276
Epoch 3/10
3424/3424 [==============================] - 0s 18us/step - loss: 0.6800 - acc: 0.6276
Epoch 4/10
3424/3424 [==============================] - 0s 26us/step - loss: 0.6673 - acc: 0.6276
Epoch 5/10
3424/3424 [==============================] - 0s 2

3425/3425 [==============================] - 0s 15us/step
Epoch 1/20
3424/3424 [==============================] - 5s 2ms/step - loss: 0.6912 - acc: 0.6025
Epoch 2/20
3424/3424 [==============================] - 0s 37us/step - loss: 0.6855 - acc: 0.6276
Epoch 3/20
3424/3424 [==============================] - 0s 28us/step - loss: 0.6756 - acc: 0.6276
Epoch 4/20
3424/3424 [==============================] - 0s 24us/step - loss: 0.6669 - acc: 0.6276
Epoch 5/20
3424/3424 [==============================] - 0s 29us/step - loss: 0.6629 - acc: 0.6276
Epoch 6/20
3424/3424 [==============================] - 0s 28us/step - loss: 0.6607 - acc: 0.6276
Epoch 7/20
3424/3424 [==============================] - 0s 27us/step - loss: 0.6598 - acc: 0.6276
Epoch 8/20
3424/3424 [==============================] - 0s 28us/step - loss: 0.6590 - acc: 0.6276
Epoch 9/20
3424/3424 [==============================] - 0s 30us/step - loss: 0.6585 - acc: 0.6276
Epoch 10/20
3424/3424 [==============================] - 0s 2

3425/3425 [==============================] - 0s 16us/step - loss: 0.6578 - acc: 0.6254
Epoch 17/20
3425/3425 [==============================] - 0s 18us/step - loss: 0.6575 - acc: 0.6254
Epoch 18/20
3425/3425 [==============================] - 0s 16us/step - loss: 0.6572 - acc: 0.6254
Epoch 19/20
3425/3425 [==============================] - 0s 16us/step - loss: 0.6570 - acc: 0.6254
Epoch 20/20
3425/3425 [==============================] - 0s 12us/step
Epoch 1/20
3425/3425 [==============================] - 5s 1ms/step - loss: 0.6907 - acc: 0.6073
Epoch 2/20
3425/3425 [==============================] - 0s 15us/step - loss: 0.6837 - acc: 0.6076
Epoch 3/20
3425/3425 [==============================] - 0s 20us/step - loss: 0.6756 - acc: 0.6076
Epoch 4/20
3425/3425 [==============================] - 0s 37us/step - loss: 0.6707 - acc: 0.6076
Epoch 5/20
3425/3425 [==============================] - 0s 29us/step - loss: 0.6697 - acc: 0.6076
Epoch 6/20
3425/3425 [==============================] - 0

Best Accuracy found is : 62.1764%   using  'batch_size': 50, 'epochs': 5